In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from google.colab import drive
import pandas as pd


epochs=[4]#4,6,8
batch_size=[4,8,12]
optimizador=["Adam","SGD","Adagrad"]
capas=[3,4,5]

#carga datos
drive.mount('/content/drive')
train_dir = '/content/drive/MyDrive/Colab Notebooks/fotos_tiempo/train'
valid_dir = '/content/drive/MyDrive/Colab Notebooks/fotos_tiempo/valid'
test_dir = '/content/drive/MyDrive/Colab Notebooks/fotos_tiempo/test'

#transformaciones
transform = transforms.Compose([
              transforms.Resize((224, 224)),
              transforms.ToTensor(),
              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
          ])


train_dataset = ImageFolder(train_dir, transform=transform)
valid_dataset = ImageFolder(valid_dir, transform=transform)
test_dataset = ImageFolder(test_dir, transform=transform)

dfError = pd.DataFrame(columns=["epocas", "batch","optimizador","capas","accuracy"])






Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
for epochs_ind in range(len(epochs)):
  for batch_ind in range(len(batch_size)):
    for opt_ind in range(len(optimizador)):
      for capas_ind in range(len(capas)):


          # Red neuronal
          class CNNWeatherClassifier3(nn.Module):
              def __init__(self, num_classes):
                  super(CNNWeatherClassifier3, self).__init__()
                  self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
                  self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
                  self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
                  self.pool = nn.MaxPool2d(2, 2)
                  self.fc1 = nn.Linear(64 * 28 * 28, 512)
                  self.fc2 = nn.Linear(512, num_classes)

              def forward(self, x):
                  x = self.pool(torch.relu(self.conv1(x)))
                  x = self.pool(torch.relu(self.conv2(x)))
                  x = self.pool(torch.relu(self.conv3(x)))
                  x = torch.flatten(x, 1)
                  x = torch.relu(self.fc1(x))
                  x = self.fc2(x)
                  return x


          class CNNWeatherClassifier4(nn.Module):
              def __init__(self, num_classes):
                  super(CNNWeatherClassifier4, self).__init__()
                  self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
                  self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
                  self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
                  self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
                  self.pool = nn.MaxPool2d(2, 2)
                  self.fc1 = nn.Linear(128 * 14 * 14, 512)
                  self.fc2 = nn.Linear(512, 256)
                  self.fc3 = nn.Linear(256, num_classes)

              def forward(self, x):
                  x = self.pool(torch.relu(self.conv1(x)))
                  x = self.pool(torch.relu(self.conv2(x)))
                  x = self.pool(torch.relu(self.conv3(x)))
                  x = self.pool(torch.relu(self.conv4(x)))
                  x = torch.flatten(x, 1)
                  x = torch.relu(self.fc1(x))
                  x = torch.relu(self.fc2(x))
                  x = self.fc3(x)
                  return x


          class CNNWeatherClassifier5(nn.Module):
            def __init__(self, num_classes):
                super(CNNWeatherClassifier5, self).__init__()
                self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
                self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
                self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
                self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
                self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
                self.pool = nn.MaxPool2d(2, 2)
                self.fc1 = nn.Linear(256 * 7 * 7, 512)
                self.fc2 = nn.Linear(512, 256)
                self.fc3 = nn.Linear(256, num_classes)

            def forward(self, x):
                x = self.pool(torch.relu(self.conv1(x)))
                x = self.pool(torch.relu(self.conv2(x)))
                x = self.pool(torch.relu(self.conv3(x)))
                x = self.pool(torch.relu(self.conv4(x)))
                x = self.pool(torch.relu(self.conv5(x)))
                x = torch.flatten(x, 1)
                x = torch.relu(self.fc1(x))
                x = torch.relu(self.fc2(x))
                x = self.fc3(x)
                return x




          train_loader = DataLoader(train_dataset, batch_size=batch_size[batch_ind], shuffle=True)
          valid_loader = DataLoader(valid_dataset, batch_size=batch_size[batch_ind], shuffle=False)
          test_loader = DataLoader(test_dataset, batch_size=batch_size[batch_ind], shuffle=False)



          #instancia del modelo
          if(capas[capas_ind]==3):
            model = CNNWeatherClassifier3(num_classes=len(train_dataset.classes))
          elif(capas[capas_ind]==4):
            model = CNNWeatherClassifier4(num_classes=len(train_dataset.classes))
          else:
            model = CNNWeatherClassifier5(num_classes=len(train_dataset.classes))



          #funcion de perdida y optimizador
          criterion = nn.CrossEntropyLoss()
          print("optimizador:")
          if(optimizador[opt_ind]=="SGD"):
            optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

          elif(optimizador[opt_ind]=="Adam"):
            optimizer = optim.Adam(model.parameters(), lr=0.001)

          else:
            optimizer = optim.Adagrad(model.parameters(), lr=0.001)

          #entrenar el modelo

          for epoch in range(epochs[epochs_ind]):
              model.train()
              running_loss = 0.0
              for images, labels in train_loader:
                  optimizer.zero_grad()
                  outputs = model(images)
                  loss = criterion(outputs, labels)
                  loss.backward()
                  optimizer.step()
                  running_loss += loss.item()
              print(f"Epoch {epoch+1}/{epochs[epochs_ind]}, Loss: {running_loss/len(train_loader)}")


          #evaluar el modelo en el conjunto de validacion
          model.eval()
          correct = 0
          total = 0
          with torch.no_grad():
              for images, labels in valid_loader:
                  outputs = model(images)
                  _, predicted = torch.max(outputs, 1)
                  total += labels.size(0)
                  correct += (predicted == labels).sum().item()

          accuracy = correct / total
          print(f"Accuracy on validation set: {accuracy}")


          #evaluacion final en el conjunto de test
          correct = 0
          total = 0
          with torch.no_grad():
              for images, labels in test_loader:
                  outputs = model(images)
                  _, predicted = torch.max(outputs, 1)
                  total += labels.size(0)
                  correct += (predicted == labels).sum().item()

          accuracy = correct / total
          print("Evaluacion final")
          print("epocas:",str(epochs[epochs_ind]))
          print("batch_size:",str(batch_size[batch_ind]))
          print("optimizador:",str(optimizador[opt_ind]))
          print("Capas:",str(capas[capas_ind]))

          print(f"Final accuracy on test set: {accuracy}")
          dfError.loc[len(dfError)] = [str(epochs[epochs_ind]),str(batch_size[batch_ind]),str(optimizador[opt_ind]), str(capas[capas_ind]), accuracy]
          print(dfError)

optimizador:
Epoch 1/4, Loss: 2.3559802919626236
Epoch 2/4, Loss: 1.816244455985725
Epoch 3/4, Loss: 1.4159871148876846
Epoch 4/4, Loss: 1.0714307319722138
Accuracy on validation set: 0.4015151515151515
Evaluacion final
epocas: 4
batch_size: 4
optimizador: Adam
Capas: 3
Final accuracy on test set: 0.42424242424242425
  epocas batch optimizador capas  accuracy
0      4     4        Adam     3  0.424242
optimizador:
Epoch 1/4, Loss: 2.3614249732345343
Epoch 2/4, Loss: 2.1860689111053944
Epoch 3/4, Loss: 2.0609410610049963
Epoch 4/4, Loss: 1.9263165909796953
Accuracy on validation set: 0.3484848484848485
Evaluacion final
epocas: 4
batch_size: 4
optimizador: Adam
Capas: 4
Final accuracy on test set: 0.3106060606060606
  epocas batch optimizador capas  accuracy
0      4     4        Adam     3  0.424242
1      4     4        Adam     4  0.310606
optimizador:
Epoch 1/4, Loss: 2.406544029712677
Epoch 2/4, Loss: 2.32377777248621
Epoch 3/4, Loss: 2.2507953103631735
Epoch 4/4, Loss: 2.1434383820

In [ ]:
dfError.to_csv('/content/drive/MyDrive/Colab Notebooks/erroresPyTorch.csv', index=False)